In [3]:
#create folder in batchs
import os, sys
path1 = 'D:\VGGface2\overlap_vggface1_2/'  #指定名称文件夹所在路径
path2 = 'D:/cnnface/female_male_test_51_addnoise/noise_0.05/'    #新建文件夹所在路径
 
def MkDir():
    dirs = os.listdir(path1)
    for dir in dirs:
        file_name = path2 + str(dir)
        os.mkdir(file_name)
        
MkDir()

In [ ]:
#extract dnn_activation
import argparse
from torchvision import transforms
from torch.utils.data import DataLoader
from dnnbrain.dnn import analyzer
from dnnbrain.dnn import io as dnn_io

stim = 'D:/cnnface/female_male_test_51_addnoise/Face_template.csv'
netloader = dnn_io.NetLoader('vggface')
imgcropsize = netloader.img_size
transform = transforms.Compose([transforms.Resize(imgcropsize),
                                    transforms.ToTensor()])  
picdataset = dnn_io.PicDataset(stim, transform=transform)
picdataloader = DataLoader(picdataset, batch_size=8, shuffle=False)
dnn_act = analyzer.dnn_activation(picdataloader, 'vggface', 'fc3')


In [ ]:
#Get classification result of network.

import torch
from torchvision import transforms

from dnnbrain.dnn.models import dnn_test_model
from dnnbrain.dnn.io import PicDataset,NetLoader,DataLoader
from cnnface.core.vgg_identity_recons import Vgg_identity

vggid = Vgg_identity()
vggid.load_state_dict(torch.load('F:/Code/pretrained_model/vgg_male_female_CrossEntro.pth'))

imgcsv_path = 'D:\cnnface/female_male_test_51_addnoise/rcicr_gabor_noise.csv'
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor()])
PicSet = PicDataset(imgcsv_path,transform)
Picloader = DataLoader(PicSet,batch_size=32)

output = dnn_test_model(Picloader,vggid)

#save label to txt file
label = output[0]
label_f = np.argwhere(label==0)
label_m = np.argwhere(label==1)

np.savetxt('D:/cnnface/female_male_test_51_addnoise/label_f.txt',label_f)
np.savetxt('D:/cnnface/female_male_test_51_addnoise/label_m.txt',label_m)

一种就是很害怕的感觉，害怕出来结果意料之外。
一个科研人不应该害怕未知，不应该自己的结果。因为实验中与理论不符，意料之外的结果才有可能掀开了真理的帘子。
开拓未知的领域，验证未知，解释未知，变成人类所掌握的技术和理论，让冰冷地客观世界变得亲切和善。
与人相伴

In [2]:
# 
import numpy as np
from dnnbrain.dnn import io
import os
from PIL import Image 
import sys
sys.path.append("..")
from core.Image_processing import average_img

csv_file = 'D:/cnnface/female_male_test_51_addnoise/rcicr_gabor_noise.csv'
index_f = np.loadtxt('D:/cnnface/female_male_test_51_addnoise/label_garbo_f.txt').astype('int64')
index_m = np.loadtxt('D:/cnnface/female_male_test_51_addnoise/label_garbo_m.txt').astype('int64')

savepath_aver_f = 'D:/cnnface/female_male_test_51_addnoise/Face_template/classification_noise/female_aver_garbo.jpg'
savepath_aver_m = 'D:/cnnface/female_male_test_51_addnoise/Face_template/classification_noise/male_aver_garbo.jpg'
savepath_CI = 'D:/cnnface/female_male_test_51_addnoise/Face_template/classification_noise/CI_garbo.jpg'

#Get pictures path from stim_csv and label txt.
picset= io.PicDataset(csv_file)
pre_picpath = picset.picpath
sub_picpath = picset.picname

f_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_f]
m_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_m]

#average all of the pictures as their label.
aver_img_f = average_img(f_picpath)
aver_img_m = average_img(m_picpath)

#calculate the CI and superimpose image and subtract image.
ci = aver_img_f - aver_img_m



(Image.fromarray(aver_img_f.astype('uint8'))).save(savepath_aver_f,quality=95)
(Image.fromarray(aver_img_m.astype('uint8'))).save(savepath_aver_m,quality=95)

In [15]:
import numpy as np
from PIL import Image

usecols = list(range(513))
usecols.remove(0)
sinusoid = np.loadtxt('D:/cnnface/rcrci/sinusoid/cycle/sinuoid_4.csv',delimiter=',',skiprows=1,usecols=usecols)
sinusoid = ((sinusoid - sinusoid.min())/(sinusoid.max() - sinusoid.min()))*255
img_sinu = Image.fromarray(sinusoid)
img_sinu.show()

In [21]:
#generateSinusoid
from PIL import Image
import numpy as np

img_size = 512
cycles = 2
angle = [0,30,60,90,120,150]
phase = [0,np.pi/2]

def generateSingleSinusoid(img_size, cycles, angle, phase):
    angle = np.radians(angle)
    sinepath = (np.linspace(0,cycles,img_size))[:,np.newaxis].repeat(img_size,axis=1)
    sinusoid = (sinepath*np.sin(angle) + sinepath.T*np.cos(angle)) * 2 *np.pi
    sinusoid = np.sin(sinusoid + phase)
    return sinusoid

def generateCombinSinusoid(img_size,cycles,angle,phase):
    sinusoid_combined = []
    for p in phase:
        for a in angle:
            sinusoid = generateSingleSinusoid(img_size,cycles,a,p)
            sinusoid = np.random.randn() * sinusoid
            sinusoid_combined.append(sinusoid)
    sinusoid_combined = np.array(sinusoid_combined)
    sinusoid_combined = np.sum(sinusoid_combined,axis=0)
    sinusoid_combined = (sinusoid_combined - sinusoid_combined.min()) / (sinusoid_combined.max() - sinusoid_combined.min())
    sinusoid_combined = sinusoid_combined *255
    return sinusoid_combined

sinusoid_combined = generateCombinSinusoid(img_size,cycles,angle,phase)
img_sinu = Image.fromarray(sinusoid_combined)
img_sinu.show()

In [19]:
print(sinusoid_combined)

[[ 0.22032912  0.22834871  0.23569744 ... -1.75558407 -1.69816976
  -1.64056921]
 [ 0.17718014  0.18622546  0.19459741 ... -1.84725798 -1.78934569
  -1.73122761]
 [ 0.13460175  0.14466297  0.15404832 ... -1.93803851 -1.87964531
  -1.82102691]
 ...
 [ 3.2101291   3.17408147  3.13693572 ...  1.41485602  1.43012029
   1.44508643]
 [ 3.16467427  3.1297722   3.09376962 ...  1.32005205  1.33594848
   1.35156671]
 [ 3.11841759  3.08467077  3.04982125 ...  1.22471651  1.24124696
   1.25751926]]
